In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.06440997123718262
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 16.06867560503086
200000 26.384181846394885
300000 28.486791900304034
400000 25.523631837931845
500000 30.17492217500399
600000 33.263394972583235
700000 41.17363013184771
800000 46.4651127802541
900000 51.728972158511475
1000000 49.510634878200044
1100000 49.66813551998845
1200000 36.653376393175414
1300000 26.296217254192065
1400000 48.17675353938278
1500000 48.204191308144445
1600000 51.88463004222597
1700000 48.40287003951516
1800000 51.11318145392866
1900000 50.34633402755319
2000000 49.376435970360234
2100000 58.931955612322334
2200000 48.40946625185842
2300000 49.31560076961781
2400000 40.20976666873704
2500000 20.116099878712323
2600000 46.78180147753956
2700000 46.0158081257647
2800000 45.37411480709848
2900000 45.360051710132396
3000000 45.791106

29700000 51.13162937556949
29800000 49.162369654807804
29900000 50.10288062247133
30000000 52.15687201277186
30100000 30.96744686442759
30200000 41.71230037243683
30300000 47.05792053551871
30400000 49.354648435500025
30500000 49.19453123762951
30600000 49.06044706511912
30700000 49.14546458903037
30800000 50.52586198643058
30900000 50.07086749945607
31000000 63.90227754347177
31100000 31.11388075283032
31200000 49.062970722889624
31300000 51.96272282063372
31400000 50.94403307869655
31500000 43.55389841404466
31600000 26.204352398094596
31700000 49.06235795244231
31800000 49.87156499162608
31900000 49.47695331841737
32000000 48.53455521106789
32100000 50.97087019861208
32200000 47.91018652930687
32300000 47.60429940689635
32400000 44.59182874728935
32500000 26.955527812085883
32600000 35.394890601566324
32700000 44.44499980443604
32800000 46.515907173476386
32900000 47.183288807191865
33000000 48.760308797947026
33100000 47.16808135656821
33200000 49.96290253808517
33300000 49.4008667

59700000 41.90245975046291
59800000 23.772883926281516
59900000 48.535614264963314
60000000 47.48840174907279
60100000 47.72674305782919
60200000 47.808791102589204
60300000 47.049507511504096
60400000 46.87875302180179
60500000 48.68360479501452
60600000 37.32656375738094
60700000 28.35835856924776
60800000 45.046580930099665
60900000 44.8466662140386
61000000 45.05958093681697
61100000 43.19977961643995
61200000 42.59730243454754
61300000 40.69972144198705
61400000 14.876572357503925
61500000 41.54795229730846
61600000 49.57278401703849
61700000 46.09915006848404
61800000 41.1024770269313
61900000 45.117234375052114
62000000 43.36671983583831
62100000 38.53695879726879
62200000 29.558455565617205
62300000 50.15033301157621
62400000 48.24726505884371
62500000 47.0015304027095
62600000 47.784467048355864
62700000 51.14786370023958
62800000 48.736800760988416
62900000 51.51084340684903
63000000 45.83687194092501
63100000 19.089629727406116
63200000 49.27062369991564
63300000 46.12144795

89800000 27.39624892068603
89900000 41.16417828545263
90000000 40.330561465960415
90100000 40.41720220230221
90200000 38.67121045964404
90300000 13.139887998413432
90400000 42.337218188863325
90500000 38.96902629516923
90600000 39.387317107644975
90700000 39.00479795246125
90800000 20.867272417833036
90900000 38.50238633591908
91000000 41.236681484363054
91100000 42.471658447212334
91200000 44.30886274972119
91300000 44.342114889244584
91400000 42.752018336954116
91500000 44.06161421882553
91600000 46.348041556882464
91700000 43.75043682059628
91800000 46.26919244268533
91900000 42.72349663413345
92000000 45.364987558309345
92100000 35.08440924354395
92200000 23.58603170505692
92300000 39.888777398770095
92400000 39.458775558643886
92500000 41.56875055006397
92600000 40.98589197665163
92700000 11.926809418201517
92800000 42.91454226941921
92900000 39.003507787347885
93000000 42.31423966741356
93100000 38.17517285691865
93200000 20.444664714525548
93300000 39.44062340684916
93400000 43.

119100000 26.223992861213745
119200000 24.912676142535418
119300000 34.896220518934236
119400000 35.495375724440656
119500000 20.980902480163234
119600000 35.725098934873614
119700000 39.25578297249581
119800000 35.3101447499924
119900000 38.47355087113884
120000000 41.30981065833169
120100000 39.86660516509832
120200000 43.963655105610314
120300000 40.64547266374818
120400000 43.0878257691629
120500000 41.914538647162715
120600000 43.33657074075278
120700000 39.84341204786305
120800000 41.030961221289665
120900000 38.32690169957997
121000000 39.654980506030206
121100000 37.68459773595582
121200000 30.657914743826613
121300000 22.676124221950463
121400000 35.70644353038827
121500000 31.74132006309459
121600000 19.548045447465046
121700000 35.66219203269953
121800000 35.51889986444537
121900000 21.44843718463986
122000000 37.796847305946926
122100000 40.57876575383644
122200000 35.96403136554111
122300000 40.82080589648939
122400000 38.32920710990394
122500000 43.99794886843981
12260000

...

494500000 4.729529402372996
494600000 5.45908839034583
494700000 7.121395665307496
494800000 6.8283086201099135
494900000 6.69186811331418
495000000 6.765577195319776
495100000 3.832143773707303
495200000 7.015919014520772
495300000 6.468399313257863
495400000 6.561131118938524
495500000 6.98271057007799
495600000 5.452060633201115
495700000 5.707079838044257
495800000 5.123580641921673
495900000 7.377069912805006
496000000 6.529576377802922
496100000 6.690697475371356
496200000 7.308040907856664
496300000 3.2701437875800203
496400000 6.294110075628393
496500000 6.470549926020903
496600000 7.245534704447393
496700000 6.208840754451257
496800000 4.872572326149805
496900000 5.990715801671827
497000000 5.767851603902531
497100000 6.883088401619727
497200000 6.357200190155503
497300000 7.228376472933329
497400000 6.616889797606211
497500000 3.315874660802133
497600000 6.13209029015649
497700000 7.251352448659087
497800000 6.149496064558622
497900000 6.204639609530396
498000000 4.3656326369

523800000 4.950081986597065
523900000 3.925896115486498
524000000 2.8193550976453583
524100000 3.528831122757913
524200000 4.824192205097336
524300000 4.83678891462137
524400000 4.745406289754193
524500000 5.494748748384326
524600000 4.9338167606392975
524700000 4.83546520769542
524800000 5.266687802104667
524900000 4.44915103537826
525000000 4.781803789043364
525100000 3.813186206628563
525200000 2.6562163319911303
525300000 3.7306397256764585
525400000 4.5947966249540855
525500000 4.52414231776803
525600000 4.635495165941372
525700000 5.286892430240181
525800000 4.982107723977958
525900000 4.8697334995468715
526000000 5.131344128745877
526100000 5.054037668711632
526200000 4.627746239128535
526300000 3.838641811895992
526400000 1.849780701728352
526500000 4.351051527794426
526600000 4.707061053959641
526700000 4.6343869035660425
526800000 4.872421975227203
526900000 5.308509446737621
527000000 5.716189941284955
527100000 5.510920028582791
527200000 6.029876534817234
527300000 5.60833

552800000 3.435046637421117
552900000 2.3904576297357267
553000000 3.523215565784934
553100000 3.6651804184634504
553200000 3.240030399775781
553300000 3.856443030894178
553400000 4.0221331652783965
553500000 4.419230152043918
553600000 3.655612422075309
553700000 2.4994795249356807
553800000 3.539593627191057
553900000 3.959232974695688
554000000 3.1887586699239368
554100000 2.2810730927419383
554200000 3.4653552965992787
554300000 3.9511723889531174
554400000 2.86216517876821
554500000 3.8852971476192684
554600000 3.9166834918440205
554700000 4.326667783777615
554800000 3.8016088594000497
554900000 2.291242018965769
555000000 3.8098019876329303
555100000 3.9047999771042035
555200000 3.117029484994199
555300000 2.093633617073646
555400000 3.454281971350441
555500000 3.958950363006793
555600000 2.6727544294741072
555700000 4.018508435199895
555800000 4.2929030688916985
555900000 3.771362746256422
556000000 3.845992139553793
556100000 2.386650809618302
556200000 4.042822119321293
556300

580300000 1.8003172622348813
580400000 1.8913601574858965
580500000 1.9061595362194579
580600000 2.7359064240921023
580700000 2.5287707668085337
580800000 1.7878372634374016
580900000 1.8172778115471802
581000000 2.069076011838579
581100000 1.8933603066499
581200000 2.539406882495412
581300000 2.501131811971023
581400000 1.9292456174607238
581500000 1.8685929053916495
581600000 1.8252075523285725
581700000 1.7291665843926873
581800000 2.4634269346829156
581900000 2.7824151700904314
582000000 1.6560000767197847
582100000 1.8851950392716867
582200000 1.8473411353333176
582300000 2.0066180716804043
582400000 2.464649845149488
582500000 2.5529268399291953
582600000 1.6449802281264545
582700000 2.1677714184178876
582800000 1.7733623873639823
582900000 1.690391882221963
583000000 2.417912561871162
583100000 2.716364904690651
583200000 1.6341740093641886
583300000 1.8927260479700208
583400000 1.7240298310830275
583500000 1.7341862066944467
583600000 2.51634176163333
583700000 2.41963741630204

608800000 0.9025124518810127
608900000 0.9138451206026673
609000000 0.8451581928544117
609100000 1.1017727818470637
609200000 0.6842923084186534
609300000 1.0377441884119583
609400000 1.0179594187948855
609500000 0.777147563646708
609600000 0.8443119088000841
609700000 1.0207327470152747
609800000 0.5961278360492468
609900000 1.018760815026076
610000000 1.0517041118006898
610100000 0.7303021894732415
610200000 0.8019680251798
610300000 0.913521981651546
610400000 0.6426488294693589
610500000 0.9360037196118897
610600000 1.0430050296362545
610700000 0.6831373051250994
610800000 0.8488783578949181
610900000 1.0404877331548108
611000000 0.5998748514168739
611100000 0.9105126409800318
611200000 0.9703369808748696
611300000 0.6321123405181288
611400000 0.7897283145729602
611500000 0.9810014381517146
611600000 0.5924001258394181
611700000 1.0176007802357665
611800000 0.9768993233593213
611900000 0.501950892528226
612000000 0.8545000342542748
612100000 0.9718898401486251
612200000 0.660032281